### Datos que contiene cada columna ###
https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

In [6]:
import pandas as pd
import matplotlib.pyplot as matplot

%matplotlib inline

# Extract data from CSV

In [7]:
taxi_dataframe = pd.read_csv('/home/jovyan/data_science_project/data/yellow_tripdata_2024-01.csv', low_memory=False)
taxi_dataframe

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/data_science_project/data/yellow_tripdata_2024-01.csv'

# Transform data

### Cambiamos el tipo de datos de las columnas de fecha

In [ ]:
taxi_dataframe['tpep_pickup_datetime'] = taxi_dataframe['tpep_pickup_datetime'].astype("datetime64[ns]")
taxi_dataframe['tpep_dropoff_datetime'] = taxi_dataframe['tpep_dropoff_datetime'].astype("datetime64[ns]")
taxi_dataframe.dtypes

: 

Eligiendo las columnas en las que estamos interesados

In [ ]:
taxi_dataframe = taxi_dataframe[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'total_amount']]

: 

In [ ]:
taxi_dataframe.head()

: 

#### Observamos los datos de manera gráfica para detectar problemas

In [ ]:
taxi_dataframe.hist(figsize=(30,20), bins=60)

: 

There are columns that need some fixing:
- RatecodeID column, according to the pdf, only have six possible ID's
- Negative values in total_amount
- Years that are not 2024 and months different from january

In [ ]:
taxi_dataframe['RatecodeID'].value_counts()

: 

Se ha marcado el número 99 unas 28663 veces como final rate code.

Tamaño total del dataframe (columnas * filas)

In [ ]:
taxi_dataframe.size

: 

#### Cantidad de datos erroneos en la columna RatecodeID

In [ ]:
wrong_ratecode = taxi_dataframe[taxi_dataframe['RatecodeID'] > 6].size
total_data = taxi_dataframe.size

wrong_ratecode_percent = (wrong_ratecode / total_data) * 100
print(f'Número de campos erroneos en ratecode: {wrong_ratecode} \nPorcentaje de errores en ratecode sobre el total de datos: {wrong_ratecode_percent:.2f}%')

: 

#### Veamos los datos nulos

In [ ]:
null_data = taxi_dataframe.isnull().sum().sum()
print(taxi_dataframe.isnull().sum())

null_percent = (null_data / total_data) * 100
print(f'Número de campos nulos: {null_data} \nPorcentaje de campos nulos sobre el total de datos: {null_percent:.2f}%')

: 

Todos los datos nulos están en la columna RatecodeID

In [ ]:
wrong_ratecode_data = wrong_ratecode_percent + null_percent
column_data = taxi_dataframe['RatecodeID'].count()

print(f'Total de datos incorrectos: {wrong_ratecode + null_data}')
print(f'Total sobre la columna: {(((wrong_ratecode + null_data) / column_data) * 100):.2f}%')

f'Porcentaje total de datos incorrectos en RatecodeID sobre el total: {wrong_ratecode_data:.2f}'

: 

#### Valores negativos en la columna total_amount.

In [ ]:
negative_values = taxi_dataframe[taxi_dataframe['total_amount'] < 0]['total_amount']
num_negative_values = negative_values.count()
negative_values_percent = (num_negative_values / total_data) * 100

column_data = taxi_dataframe['total_amount'].count()
print(column_data)
negative_values_column = (num_negative_values / column_data) * 100

print(f"Porcentaje de valores negativos en 'total_amount' sobre el total del DataFrame: {negative_values_percent:.2f}%")
print(f"Porcentaje de valores negativos en 'total_amount' sobre la columna: {negative_values_column:.2f}%")

: 

In [ ]:
taxi_dataframe.dtypes

: 

Procedo a eliminar los datos erroneos de RatecodeID, borrar los datos negativos de la columna total_amount y quitar los campos nulos.

In [ ]:
taxi_dataframe_filtered = taxi_dataframe[(taxi_dataframe['RatecodeID'] <= 6) & (taxi_dataframe['total_amount'] > 0)]

taxi_dataframe_filtered = taxi_dataframe_filtered.dropna()

print('''Comprobamos que se han limpiado los datos''')
print(taxi_dataframe_filtered['RatecodeID'].value_counts())
print('''Datos negativos en total_amount''')
print(taxi_dataframe_filtered[taxi_dataframe_filtered['total_amount'] < 0].shape)
print(f'Campos nulos: \n{taxi_dataframe_filtered.isnull().sum()}')

: 

In [ ]:
print(f'Datos originales: {taxi_dataframe.shape}')

print(f'Datos filtrados: {taxi_dataframe_filtered.shape}')

: 

In [ ]:
taxi_dataframe_filtered.dtypes

: 

Las columnas RatecodeID, PULocationID, DOLocationID, payment type las podemos cambiar a un tipo diferente de númerico ya que por ejemplo RatecodeID y paymen_type se refieren a la tarifa aplicada al final del viaje representada numericamente y de manera similar con el payment_type que se refiere a la forma de pago.
Las columnas de Location se refieren a zonas o barrios, así que no son datos numericos perse.

In [ ]:
for column in taxi_dataframe_filtered.columns:
    if 'ID' in column or 'payment_type' in column:
        taxi_dataframe_filtered.loc[:,column] = taxi_dataframe_filtered[column].astype('str')


: 

In [ ]:
taxi_dataframe_filtered.dtypes

: 

In [ ]:
taxi_dataframe_filtered

: 

#### Volvemos a ver los gráficos de datos

In [ ]:
taxi_dataframe_filtered.hist(figsize=(30,20), bins=60)

: 

### Se puede ver que aún hay problemas con los datos

Hay años diferentes a 2024 y meses diferentes a enero

In [ ]:
not_2024 = taxi_dataframe_filtered.loc[
  taxi_dataframe_filtered['tpep_pickup_datetime'].dt.year < 2024, ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
  ]

not_january = taxi_dataframe_filtered.loc[
  taxi_dataframe_filtered['tpep_pickup_datetime'].dt.month > 1, 
  ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
  ]

print(f"Años diferentes a 2024: \n {not_2024} \n")

print(f"Meses distintos de enero: \n{not_january} \n")

: 

### Queremos datos exclusivamente de 2024 y del mes de enero.
He creado una nueva variable "taxi_data_prep" que va a contener el nuevo DataFrame sin años menores a 2024 o meses diferentes a enero

In [ ]:
taxi_data_prep = taxi_dataframe_filtered.drop(
  taxi_dataframe_filtered[
  (taxi_dataframe_filtered['tpep_pickup_datetime'].dt.year != 2024)
  ].index)

taxi_data_prep = taxi_data_prep.drop(
  taxi_data_prep[
    (taxi_data_prep['tpep_pickup_datetime'].dt.month > 1) |
    (taxi_dataframe_filtered['tpep_dropoff_datetime'].dt.month > 1)
    ].index)

print('Viajes que inician en un mes diferente a enero')
print(taxi_data_prep.loc[
  ((taxi_data_prep['tpep_pickup_datetime'].dt.month > 1) &
    (taxi_dataframe_filtered['tpep_dropoff_datetime'].dt.month > 1)),
  ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
  ])

print('Años diferentes de 2024')
print(taxi_data_prep.loc[
  (taxi_data_prep['tpep_pickup_datetime'].dt.year != 2024), 
  # &(taxi_dataframe_filtered['tpep_pickup_datetime'].dt.month > 1),
  ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
  ])

: 

#### Datos trip_distance y total_amount no concordantes.
Hay datos en ambas columnas que son absurdos y no correlacionan.
Vamos a limpiar esos datos

In [ ]:
taxi_data_prep.loc[
  (taxi_data_prep['trip_distance'] > 250) | (taxi_data_prep['total_amount'] > 200)
  ]

: 

In [ ]:
taxi_data_prep = taxi_data_prep.drop(
  taxi_data_prep[
  (taxi_data_prep['trip_distance'] > 250) | 
  (taxi_data_prep['trip_distance'] == 0.0)
  ].index)

taxi_data_prep.loc[(taxi_data_prep['trip_distance'] > 250) | 
  (taxi_data_prep['trip_distance'] == 0.0)]

: 

In [ ]:
taxi_data_prep.loc[
  (taxi_data_prep['trip_distance'] > 250) | (taxi_data_prep['total_amount'] > 200)
  ].value_counts()

: 

### Calcular el promedio de gasto del mes de enero

In [ ]:
average_spent_january = taxi_data_prep['total_amount'].mean()
print(f'Media de gasto en el mes de enero: {average_spent_january:.2f}')

mode_january = taxi_data_prep['total_amount'].mode().to_string()

print(f'El gasto que más aparece en el mes de enero: {mode_january[-4:]}')

: 

### Calcular el promedio por día

In [ ]:
daily_totals = taxi_data_prep.groupby(taxi_data_prep['tpep_pickup_datetime'].dt.day)['total_amount'].mean()

# Calcular el número de días en enero
num_days = len(daily_totals)
print(num_days)

# Calcular el gasto promedio diario
average_daily_spent = daily_totals.mean()
print(f"El gasto promedio diario en enero de 2024 es: {average_daily_spent:.2f}")

: 

# TODO: El pickup location se puede dejar como númerico y con mode se puede ver qué zona se repite más

In [ ]:
taxi_data_prep.hist(figsize=(30,20), bins=60)

: 

: 